# BSc Apprenticeship Degree in Digital Technology & Solutions
#### Author
Zach Molony
#### Module Name                  
Data Structure & Algorithms
#### Objectives
- Empirical analysis of sorting algorithms using Big-O notation.
- Empirical analysis of search algorithms using Big-O notation.
- Design of recursive method for problem solving.

In [1]:
# Imports
%matplotlib inline

from random import sample
import timeit
from functools import partial as wrapper
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.mlab as mlab
import csv

## PART I: Empirical Analysis of Sorting Algorithms (50%)
### Algorithm code

In [2]:
# Bubble sort
def bubble(arr):
    n = len(arr)
    for i in range(n):
        for x in range(1, n):
            if arr[x] < arr[x-1]:
                arr[x], arr[x-1] = arr[x-1], arr[x]
    return arr

# Insertion sort
def insertion(arr):
    for i in range(1, len(arr)):
        current = arr[i]
        while i > 0 and arr[i-1]>current:
            arr[i] = arr[i-1]
            i = i - 1
            arr[i] = current
    return arr

# Selection sort
def selection(arr):  
    for i in range(len(arr)): 
        min_idx = i 
        for j in range(i + 1, len(arr)): 
            if arr[min_idx] > arr[j]: 
                min_idx = j
        arr[i], arr[min_idx] = arr[min_idx], arr[i] 
    return arr

# Merge sort
def merge(arr):
    if len(arr) > 1: 
        mid = len(arr) // 2
        leftArr = arr[:mid]
        rightArr = arr[mid:]
        merge(leftArr)
        merge(rightArr)
        i = j = k = 0
        while i < len(leftArr) and j < len(rightArr): 
            if leftArr[i] < rightArr[j]: 
                arr[k] = leftArr[i] 
                i += 1
            else: 
                arr[k] = rightArr[j] 
                j += 1
            k += 1
        while i < len(leftArr): 
            arr[k] = leftArr[i] 
            i += 1
            k += 1
        while j < len(rightArr): 
            arr[k] = rightArr[j] 
            j += 1
            k += 1
    return arr

# Shell sort
def shell(arr):
    n = len(arr) 
    gap = n // 2
    while gap > 0: 
        for i in range(gap,n):
            temp = arr[i]
            j = i 
            while  j >= gap and arr[j-gap] > temp: 
                arr[j] = arr[j-gap] 
                j -= gap 
            arr[j] = temp 
        gap //= 2
    return arr

In [3]:
# Define constants (master object for all timings / list sizes / lists to be generated)
master = {}
inputSizes = [1000, 5000, 10000, 25000, 50000, 75000, 100000]
# Bigger lists for the more efficient algorithms
inputSizesBigger = [1000, 5000, 10000, 25000, 50000, 100000, 500000, 1000000, 5000000, 10000000]

#Objects to hold actual arrays
randomLists = {}
randomListsBigger = {}

In [4]:
# Create lists of random numbers using random.sample
for size in inputSizes:
    randomLists[str(size)] = sample(range(0, size+1), size)

for size in inputSizesBigger:
    randomListsBigger[str(size)] = sample(range(0, size+1), size)

### Timing function
Here I am measuring the timing of the algorithms using the timeit.Timer() function. This measures actual wall time and therefore it must be acknowledged that all measurements have been tested on the same system under the same conditions:
- Tested on an Intel(R) Core(TM) i5-4690K CPU OC @ 4.0GHz.

This function takes one parameter, which is the number of times the program is run for it to take an average from: ```t.timeit(n)```. Where ```n``` is the amount repetitions averaged. I am running each function 3 times as this will give me an accurate average while not taking a day. 😉

I am using the ```'wrapper'``` function to insert the array into the function - this is an issue with the timer function.

**Some notes:** Sadly, I couldn't work out how to create a single function which could run the below code on all the functions, this was due to the timer I chose not allowing multiple functions with parameters to be passed in. I decided to stay with this implementation as it still is accurate and effective, although the below section of the code needs to be repeated for each algorithm.

Alternately, this could have been measured in operations, adding one to a counter each time a comparison was made. I unfortunately did not have time to implement this additionally however I may add a proof of concept at the end of this section. 

In [5]:
# Init object to record times
times = {}

# For every size, create a timer 
for size in randomLists:
    t = timeit.Timer(wrapper(bubble, randomLists[size]))
    times[size] = round(t.timeit(3), 6)
    
# Add to master object
master['bubble'] = times

KeyboardInterrupt: 

In [ ]:
times = {}
for size in randomLists:
    t = timeit.Timer(wrapper(selection, randomLists[size]))
    times[size] = round(t.timeit(3), 6)
master['selection'] = times

In [5]:
times = {}
for size in randomLists:
    print(len(randomLists[size]))
    t = timeit.Timer(wrapper(insertion, randomLists[size]))
    times[size] = round(t.timeit(1), 6)
master['insertion'] = times

1000
5000
10000
25000
50000
75000
100000


In [ ]:
times = {}
for size in randomListsBigger:
    t = timeit.Timer(wrapper(merge, randomListsBigger[size]))
    times[size] = round(t.timeit(3), 6)
master['merge'] = times

In [ ]:
times = {}
for size in randomListsBigger:
    t = timeit.Timer(wrapper(shell, randomListsBigger[size]))
    times[size] = round(t.timeit(3), 6)
master['shell'] = times

In [10]:
times = {}
t = timeit.Timer(wrapper(shell, randomListsBigger['10000000']))
times[size] = round(t.timeit(3), 6)
master['shell'] = times

### Write to CSV file
Create a backup of the timings external the the projects. All variables in this notebook are volatile so if I return after closing it I will need to rerun the lengthy timing functions.

In [11]:
# Write to CSV

with open('timingsshell.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for obj in master:
        writer.writerow([obj] + [time for time in master[obj].values()])

# Analysis
Now you have seen how the measurements have been generated and stored, in this section I will begin my analysis on the time complexity of the sorting algorithms. Below I am reading the times from the file I have previously made for the reasons explained previously.

In [ ]:
readMaster = {} #create a new, constant master as the variables in this notebook are volatile 
with open('timings.csv') as timings:
    readCSV = csv.reader(timings, delimiter=',')
    for row in readCSV:
        readMaster[row[0]] = row[1:]

## Bubble Sort
Bubble sort works by iterating through a list of numbers, and then for each number, comparing it with subsequent numbers until it is in the correct place. Clearly as each number in the list is compared with each other number in the list, the time complexity will be $O(n^{2})$.

Let us manually calculate the time complexity in a line by line analysis:

In [ ]:
def bubble(arr):                                      # Operations | Subtotal | Total
    n = len(arr)                                      # 1 + 1       = 1  
    for i in range(n):                                # n + 1 + 1   = n
        for x in range(1, n):                         # n + 1 + 1   = n  
            if arr[x] < arr[x-1]:                     # 1 + 1       = 1
                arr[x], arr[x-1] = arr[x-1], arr[x]   # 1 + 1 + 1   = 1
                                                      #----------------------------
                                                      # Total       = n * n    = n^2

This implementation of bubble sort has a complexity of $О(n^{2})$, where n is the number of items being sorted. This is the worst class of time complexity out of the algorithms in this report. The expected time complexity should be around $O(n log n)$, furthermore, of the other $О(n^{2})$ sorting algorithms bubble is by far the slowest, as we can see from the graph below, illustrating extremely high operation time counts.

In [ ]:
x = inputSizes
bub = [float(t) for t in readMaster['bubble']]
sel = [float(t) for t in readMaster['selection']]
ins = [float(t) for t in readMaster['insertion']][:-3]

plt.plot(x, bub, color='red', linestyle='solid', linewidth = 1, marker='o', markersize=3) 
plt.plot(x, sel, color='blue', linestyle='solid', linewidth = 1, marker='o', markersize=3) 
plt.plot(x, ins, color='green', linestyle='solid', linewidth = 1, marker='o', markersize=3) 
plt.xlabel('Array Size') 
plt.ylabel('Seconds') 
plt.title('O(n^2) Algorithms')
plt.show() 

In [ ]:
x = inputSizes
y = [float(t) for t in readMaster['bubble']]

# plotting the points  
plt.plot(x, y, color='green', linestyle='solid', linewidth = 1, marker='o', markerfacecolor='blue', markersize=4) 

# naming the x axis 
plt.xlabel('Array Size') 
# naming the y axis 
plt.ylabel('Seconds') 
  
# giving a title to my graph 
plt.title('Bubble sort') 
  
# function to show the plot 
plt.show() 

### Improvements?

We can clearly see that bubble sort struggles with large data sets, so perhaps we can optimise this somehow. See the below implementation. This iteration includes a flag, which improves the performance slightly: for an already sorted list, the array is still $O(n^{2})$ dispite no swaps being made. With this fix, the performance is now $O(n)$ in the best case Scenario, as the array will complete one iteration of the inner loop and break.

In [ ]:
# Bubble sort
def bubble(arr): 
    n = len(arr) 
    # iterate through the the array
    for i in range(n): 
        swapped = False # create a flag to check if a swap has occurred
        for j in range(0, n-i-1): # final i elements will already be in place so no need to check them
            # perform a swap if the elements are out of order
            if arr[j] > arr[j+1]: 
                arr[j], arr[j+1] = arr[j+1], arr[j] 
                swapped = True
        if swapped == False: # there is no need to keep iterating as the array is in order
            break
    return arr

In conclusion, this modification improves the best case time to $O(n)$, and slightly improves the runtime of the overall algorithm and thereby the average case, although still remains at $O(n^{2})$. This in some ways is better than much more complex algorithms as they can sometimes be slower for a fully sorted list. Still though, even with this improvement, another $O(n^{2})$ algorithm such as insertion sort is far superior as it shares this best case, it is still much faster in it's average time, as depicted by the graph previously mentioned.

## Selection Sort
Selection sort is another extremely simple sorting algorithm which runs two  

In [ ]:
x = inputSizes
y = [float(t) for t in readMaster['selection']]

plt.plot(x, y, color='green', linestyle='solid', linewidth = 1, marker='o', markerfacecolor='blue', markersize=4) 
plt.xlabel('Array Size')
plt.ylabel('Seconds') 
plt.title('Selection sort')
plt.show() 

## Insertion Sort

In [ ]:
x = np.array(inputSizesBigger)
y = np.array([float(t) for t in master['insertion']])

plt.plot(x, y, color='green', linestyle='solid', linewidth = 1, marker='o', markerfacecolor='blue', markersize=4) 
plt.xscale('log')
plt.xlabel('Array Size')
plt.ylabel('Seconds') 
plt.title('Insertion sort')
plt.show() 

## Merge Sort

In [ ]:
x = inputSizesBigger
y = [float(t) for t in readMaster['merge']]

plt.plot(x, y, color='green', linestyle='solid', linewidth = 1, marker='o', markerfacecolor='blue', markersize=4) 
plt.xscale('log')
#plt.yscale('log')
plt.xlabel('Array Size')
plt.ylabel('Seconds') 
plt.title('Merge sort')
plt.show() 

## Shell Sort

In [ ]:
x = inputSizesBigger
y = [float(t) for t in master['shell']]

plt.plot(x, y, color='green', linestyle='solid', linewidth = 1, marker='o', markerfacecolor='blue', markersize=4) 
plt.xscale('log')
plt.xlabel('Array Size')
plt.ylabel('Seconds') 
plt.title('Shell sort')
plt.show() 